# **산불 데이터 전처리**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#경로 설정 
import os
os.chdir('/content/drive/MyDrive/A2W/데이터')

In [ ]:
pip install pyproj

     |████████████████████████████████| 6.6MB 12.9MB/s 


In [ ]:
pip install rtree

     |████████████████████████████████| 1.0MB 13.3MB/s 


In [ ]:
pip install geopandas

     |████████████████████████████████| 1.0MB 13.2MB/s 
     |████████████████████████████████| 15.4MB 254kB/s 


In [ ]:
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string
import matplotlib.font_manager as fm
import matplotlib 

In [ ]:
fire_forest = gpd.GeoDataFrame.from_file("Forest_fire_ALL.shp", encoding = 'euc-kr')

In [ ]:
fire_forest

,OCUR_DTM,OCUR_YOIL,EXTING_DT,RQRMN_TM,OCUR_GM,OCUR_DO,OCUR_SGG,OCUR_EMD,OCUR_RI,OCUR_JIBUN,OCUR_CAUSE,DMG_AREA,DMG_MONEY,TM_X,TM_Y,geometry
0,201303231155,토,201303231630,0435,북부,경기,양평군,서종면,수입리,산16,담뱃불실화,None,None,236837.618025178,460698.20470856,POINT (992784.224 1960940.446)
1,201303231438,토,201303231540,0102,강원,강원,원주시,지정면,안창리,산19-1,성묘객실화,None,None,269720.71082044,428262.436219449,POINT (1025480.619 1928345.386)
2,201303241630,일,201303241820,0150,전남,전남,진도군,조도면,신육리,산278,낚시꾼실화,None,None,112940.739111932,87018.1903259957,POINT (867023.972 1588030.309)
3,201303251315,월,201303251420,0105,경북,경북,청도군,금천면,박곡리,산61,쓰레기소각,None,None,374158.094638993,244058.50040305,POINT (1128909.380 1743698.587)
4,201303260930,화,201303261020,0050,전북,전북,장수군,장수읍,덕산리,산54-1,입산자실화,None,None,251187.949237867,234084.370629366,POINT (1005952.502 1734352.893)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9980,199705011900,목,199705012030,0130,충남,충남,계룡,두마,입암,산73-1,입산자실화추정,0.07,0,222038.567649,304235.36542,POINT (977173.715 1804622.087)
9981,199704130950,일,199704131420,0430,경북,경북,포항,송라,방석,산35-1,논두렁소각,4.5,900,412156.770147,306944.212845,POINT (1167206.182 1806351.941)
9982,201005071456,금,201005071625,0129,중부,충북,옥천,청성,양저,산29-3,산나물채취자실화추정,0.1,0,265529.4918,311044.5097,POINT (1020680.378 1811204.463)
9983,201003291530,월,201003291644,0114,경북,경북,청송,안덕,장전,산4,입산자실화,0.3,5120,377476.0375,312470.1011,POINT (1132575.033 1812052.937)


In [ ]:
fire_forest.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 9985 entries, 0 to 9984
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OCUR_DTM    9985 non-null   object  
 1   OCUR_YOIL   9809 non-null   object  
 2   EXTING_DT   9809 non-null   object  
 3   RQRMN_TM    9807 non-null   object  
 4   OCUR_GM     9942 non-null   object  
 5   OCUR_DO     9985 non-null   object  
 6   OCUR_SGG    9985 non-null   object  
 7   OCUR_EMD    9985 non-null   object  
 8   OCUR_RI     7362 non-null   object  
 9   OCUR_JIBUN  9934 non-null   object  
 10  OCUR_CAUSE  9743 non-null   object  
 11  DMG_AREA    9691 non-null   object  
 12  DMG_MONEY   9691 non-null   object  
 13  TM_X        9985 non-null   object  
 14  TM_Y        9985 non-null   object  
 15  geometry    9985 non-null   geometry
dtypes: geometry(1), object(15)
memory usage: 1.2+ MB


In [ ]:
fire_forest["date"]=fire_forest["EXTING_DT"].str[0:8]
fire_forest["date"] = fire_forest["date"].astype('datetime64[ns]')
fire_forest.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 9985 entries, 0 to 9984
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   OCUR_DTM    9985 non-null   object        
 1   OCUR_YOIL   9809 non-null   object        
 2   EXTING_DT   9809 non-null   object        
 3   RQRMN_TM    9807 non-null   object        
 4   OCUR_GM     9942 non-null   object        
 5   OCUR_DO     9985 non-null   object        
 6   OCUR_SGG    9985 non-null   object        
 7   OCUR_EMD    9985 non-null   object        
 8   OCUR_RI     7362 non-null   object        
 9   OCUR_JIBUN  9934 non-null   object        
 10  OCUR_CAUSE  9743 non-null   object        
 11  DMG_AREA    9691 non-null   object        
 12  DMG_MONEY   9691 non-null   object        
 13  TM_X        9985 non-null   object        
 14  TM_Y        9985 non-null   object        
 15  geometry    9985 non-null   geometry      
 16  date        9809

In [ ]:
fire_forest = fire_forest.loc[:, ['date', 'OCUR_DO', 'OCUR_SGG', 'OCUR_EMD', 'geometry']]
fire_forest

,date,OCUR_DO,OCUR_SGG,OCUR_EMD,geometry
0,2013-03-23,경기,양평군,서종면,POINT (992784.224 1960940.446)
1,2013-03-23,강원,원주시,지정면,POINT (1025480.619 1928345.386)
2,2013-03-24,전남,진도군,조도면,POINT (867023.972 1588030.309)
3,2013-03-25,경북,청도군,금천면,POINT (1128909.380 1743698.587)
4,2013-03-26,전북,장수군,장수읍,POINT (1005952.502 1734352.893)
...,...,...,...,...,...
9980,1997-05-01,충남,계룡,두마,POINT (977173.715 1804622.087)
9981,1997-04-13,경북,포항,송라,POINT (1167206.182 1806351.941)
9982,2010-05-07,충북,옥천,청성,POINT (1020680.378 1811204.463)
9983,2010-03-29,경북,청송,안덕,POINT (1132575.033 1812052.937)


In [ ]:
fire_forest_gs = fire_forest[(fire_forest["OCUR_DO"] == "경남") | (fire_forest["OCUR_DO"] == "경북")]
fire_forest_gs

,date,OCUR_DO,OCUR_SGG,OCUR_EMD,geometry
3,2013-03-25,경북,청도군,금천면,POINT (1128909.380 1743698.587)
6,2013-03-26,경북,경산시,용성면,POINT (1121618.562 1757258.233)
14,2013-03-28,경남,양산시,상북면,POINT (1141119.617 1716463.298)
15,2013-03-29,경남,의령군,용덕면,POINT (1069869.270 1707304.278)
18,2013-03-29,경남,사천시,신수동,POINT (1052896.224 1656085.584)
...,...,...,...,...,...
9976,2009-10-11,경북,구미,고아,POINT (1073154.317 1799683.834)
9977,2010-01-11,경북,포항,청하,POINT (1167973.063 1800439.209)
9978,2002-06-19,경북,포항,죽장,POINT (1137989.731 1799626.268)
9981,1997-04-13,경북,포항,송라,POINT (1167206.182 1806351.941)


In [ ]:
fire_forest_gs = fire_forest_gs.sort_values(by=['date'], ignore_index=True)
fire_forest_gs

,date,OCUR_DO,OCUR_SGG,OCUR_EMD,geometry
0,1966-01-22,경남,밀양,무안,POINT (1100709.547 1720742.207)
1,1984-01-08,경북,포항,북,POINT (1167381.185 1784764.180)
2,1984-03-18,경북,포항,흥해,POINT (1161475.245 1792041.736)
3,1984-03-25,경남,양산,평산,POINT (1149123.517 1710471.619)
4,1984-03-27,경북,안동,임동,POINT (1127552.184 1846410.704)
...,...,...,...,...,...
2604,NaT,경남,하동,옥종,POINT (1030355.919 1688240.518)
2605,NaT,경남,하동,청암,POINT (1024927.741 1687246.966)
2606,NaT,경남,하동,북천,POINT (1034034.773 1680777.879)
2607,NaT,경남,하동,옥종,POINT (1035061.306 1692213.394)


In [ ]:
fire_forest_gs = fire_forest_gs.dropna(subset=['date'])
fire_forest_gs

,date,OCUR_DO,OCUR_SGG,OCUR_EMD,geometry
0,1966-01-22,경남,밀양,무안,POINT (1100709.547 1720742.207)
1,1984-01-08,경북,포항,북,POINT (1167381.185 1784764.180)
2,1984-03-18,경북,포항,흥해,POINT (1161475.245 1792041.736)
3,1984-03-25,경남,양산,평산,POINT (1149123.517 1710471.619)
4,1984-03-27,경북,안동,임동,POINT (1127552.184 1846410.704)
...,...,...,...,...,...
2515,2013-11-20,경북,영덕군,영덕읍,POINT (1165514.147 1830672.247)
2516,2013-11-21,경북,포항,남,POINT (1179725.078 1768405.329)
2517,2013-12-04,경북,영천시,화산면,POINT (1119623.324 1789418.727)
2518,2013-12-27,경남,김해시,대동면,POINT (1130065.262 1696983.796)


In [ ]:
# 2000년도 이후 데이터만 저장
fire_forest_gs_test = fire_forest_gs.iloc[761:, :]
fire_forest_gs_test

,date,OCUR_DO,OCUR_SGG,OCUR_EMD,geometry
761,2000-01-01,경북,칠곡,북삼,POINT (1072981.132 1787464.008)
762,2000-01-21,경북,영천,신령,POINT (1114379.412 1786272.905)
763,2000-01-27,경북,군위,군위,POINT (1099359.822 1808565.571)
764,2000-01-28,경북,구미,해평,POINT (1084301.730 1802107.630)
765,2000-01-30,경북,경주,감포,POINT (1176170.073 1759529.982)
...,...,...,...,...,...
2515,2013-11-20,경북,영덕군,영덕읍,POINT (1165514.147 1830672.247)
2516,2013-11-21,경북,포항,남,POINT (1179725.078 1768405.329)
2517,2013-12-04,경북,영천시,화산면,POINT (1119623.324 1789418.727)
2518,2013-12-27,경남,김해시,대동면,POINT (1130065.262 1696983.796)


In [ ]:
# 2011년도 이후 데이터만 저장
fire_forest_gs_test = fire_forest_gs.iloc[2294:, :]
fire_forest_gs_test

,date,OCUR_DO,OCUR_SGG,OCUR_EMD,geometry
2294,2011-01-01,경남,거제,일운,POINT (1107951.350 1649374.010)
2295,2011-01-01,경남,거제,일운,POINT (1109865.639 1650286.486)
2296,2011-01-01,경남,함안,칠북,POINT (1095089.445 1708344.396)
2297,2011-01-08,경남,산청,단성,POINT (1037468.228 1700561.163)
2298,2011-01-11,경남,산청,생비량,POINT (1050328.776 1703704.330)
...,...,...,...,...,...
2515,2013-11-20,경북,영덕군,영덕읍,POINT (1165514.147 1830672.247)
2516,2013-11-21,경북,포항,남,POINT (1179725.078 1768405.329)
2517,2013-12-04,경북,영천시,화산면,POINT (1119623.324 1789418.727)
2518,2013-12-27,경남,김해시,대동면,POINT (1130065.262 1696983.796)


In [ ]:
# 행정동 경계 파일 읽기
hangjung_gubun = gpd.GeoDataFrame.from_file("umd.shp", encoding = 'utf-8')
hangjung_gubun

,BASE_DATE,ADM_DR_CD,ADM_DR_NM,geometry
0,20200630,3701111,구룡포읍,"MULTIPOLYGON (((1184734.239 1774617.254, 11847..."
1,20200630,3701112,연일읍,"POLYGON ((1160923.832 1786524.018, 1160962.498..."
2,20200630,3701113,오천읍,"POLYGON ((1172424.168 1778414.742, 1172426.383..."
3,20200630,3701131,대송면,"POLYGON ((1167538.928 1778439.770, 1167541.585..."
4,20200630,3701132,동해면,"MULTIPOLYGON (((1181368.959 1784712.163, 11813..."
...,...,...,...,...
632,20200630,3840042,쌍백면,"POLYGON ((1056287.745 1722189.578, 1056335.116..."
633,20200630,3840043,삼가면,"POLYGON ((1052540.942 1719533.002, 1052573.371..."
634,20200630,3840044,가회면,"POLYGON ((1044199.527 1722837.244, 1044218.563..."
635,20200630,3840045,대병면,"POLYGON ((1046756.129 1730112.827, 1046808.008..."


In [ ]:
# 행정동 시 경계 파일 읽기
hangjung_gubun_si = pd.read_csv('umd_si.csv', encoding='utf-8')

In [ ]:
hangjung_gubun["sgg"] = hangjung_gubun_si

In [ ]:
hangjung_gubun

,BASE_DATE,ADM_DR_CD,ADM_DR_NM,geometry,sgg
0,20200630,3701111,구룡포읍,"MULTIPOLYGON (((1184734.239 1774617.254, 11847...",포항시 남구
1,20200630,3701112,연일읍,"POLYGON ((1160923.832 1786524.018, 1160962.498...",포항시 남구
2,20200630,3701113,오천읍,"POLYGON ((1172424.168 1778414.742, 1172426.383...",포항시 남구
3,20200630,3701131,대송면,"POLYGON ((1167538.928 1778439.770, 1167541.585...",포항시 남구
4,20200630,3701132,동해면,"MULTIPOLYGON (((1181368.959 1784712.163, 11813...",포항시 남구
...,...,...,...,...,...
632,20200630,3840042,쌍백면,"POLYGON ((1056287.745 1722189.578, 1056335.116...",합천군
633,20200630,3840043,삼가면,"POLYGON ((1052540.942 1719533.002, 1052573.371...",합천군
634,20200630,3840044,가회면,"POLYGON ((1044199.527 1722837.244, 1044218.563...",합천군
635,20200630,3840045,대병면,"POLYGON ((1046756.129 1730112.827, 1046808.008...",합천군


In [ ]:
result = gpd.sjoin(hangjung_gubun, fire_forest_gs_test, how='right', op='intersects')
result

,index_left,BASE_DATE,ADM_DR_CD,ADM_DR_NM,sgg,date,OCUR_DO,OCUR_SGG,OCUR_EMD,geometry
2294,426.0,20200630,3809031,일운면,거제시,2011-01-01,경남,거제,일운,POINT (1107951.350 1649374.010)
2295,426.0,20200630,3809031,일운면,거제시,2011-01-01,경남,거제,일운,POINT (1109865.639 1650286.486)
2296,532.0,20200630,3832036,칠북면,함안군,2011-01-01,경남,함안,칠북,POINT (1095089.445 1708344.396)
2297,593.0,20200630,3837037,단성면,산청군,2011-01-08,경남,산청,단성,POINT (1037468.228 1700561.163)
2298,595.0,20200630,3837039,생비량면,산청군,2011-01-11,경남,산청,생비량,POINT (1050328.776 1703704.330)
...,...,...,...,...,...,...,...,...,...,...
2515,253.0,20200630,3735011,영덕읍,영덕군,2013-11-20,경북,영덕군,영덕읍,POINT (1165514.147 1830672.247)
2516,5.0,20200630,3701133,장기면,포항시 남구,2013-11-21,경북,포항,남,POINT (1179725.078 1768405.329)
2517,147.0,20200630,3707033,화산면,영천시,2013-12-04,경북,영천시,화산면,POINT (1119623.324 1789418.727)
2518,397.0,20200630,3807037,대동면,김해시,2013-12-27,경남,김해시,대동면,POINT (1130065.262 1696983.796)


In [ ]:
result.isnull().sum()

index_left    3
BASE_DATE     3
ADM_DR_CD     3
ADM_DR_NM     3
sgg           3
date          0
OCUR_DO       0
OCUR_SGG      0
OCUR_EMD      0
geometry      0
dtype: int64

In [ ]:
result_null = result[result["BASE_DATE"].isnull()]
result_null

,index_left,BASE_DATE,ADM_DR_CD,ADM_DR_NM,sgg,date,OCUR_DO,OCUR_SGG,OCUR_EMD,geometry
2396,NaN,NaN,NaN,NaN,NaN,2012-03-28,경남,영광,불갑면,POINT (909496.685 1690055.214)
2428,NaN,NaN,NaN,NaN,NaN,2012-06-06,경남,양산,물금읍,POINT (920770.377 1797473.031)
2451,NaN,NaN,NaN,NaN,NaN,2013-03-04,경남,사천시,축동,POINT (926857.134 1692317.457)


In [ ]:
fire_forest_gs_test1 = fire_forest_gs_test[(fire_forest_gs_test["OCUR_EMD"] != "불갑면") & (fire_forest_gs_test["OCUR_EMD"] != "물금읍") & (fire_forest_gs_test["OCUR_EMD"] != "축동")]
fire_forest_gs_test1

,date,OCUR_DO,OCUR_SGG,OCUR_EMD,geometry
2294,2011-01-01,경남,거제,일운,POINT (1107951.350 1649374.010)
2295,2011-01-01,경남,거제,일운,POINT (1109865.639 1650286.486)
2296,2011-01-01,경남,함안,칠북,POINT (1095089.445 1708344.396)
2297,2011-01-08,경남,산청,단성,POINT (1037468.228 1700561.163)
2298,2011-01-11,경남,산청,생비량,POINT (1050328.776 1703704.330)
...,...,...,...,...,...
2515,2013-11-20,경북,영덕군,영덕읍,POINT (1165514.147 1830672.247)
2516,2013-11-21,경북,포항,남,POINT (1179725.078 1768405.329)
2517,2013-12-04,경북,영천시,화산면,POINT (1119623.324 1789418.727)
2518,2013-12-27,경남,김해시,대동면,POINT (1130065.262 1696983.796)


In [ ]:
result1 = gpd.sjoin(hangjung_gubun, fire_forest_gs_test1, how='left', op='intersects')
result1

,BASE_DATE,ADM_DR_CD,ADM_DR_NM,geometry,sgg,index_right,date,OCUR_DO,OCUR_SGG,OCUR_EMD
0,20200630,3701111,구룡포읍,"MULTIPOLYGON (((1184734.239 1774617.254, 11847...",포항시 남구,NaN,NaT,NaN,NaN,NaN
1,20200630,3701112,연일읍,"POLYGON ((1160923.832 1786524.018, 1160962.498...",포항시 남구,2464.0,2013-03-10,경북,포항시,연일읍
2,20200630,3701113,오천읍,"POLYGON ((1172424.168 1778414.742, 1172426.383...",포항시 남구,2503.0,2013-05-13,경북,포항시남구,오천읍
3,20200630,3701131,대송면,"POLYGON ((1167538.928 1778439.770, 1167541.585...",포항시 남구,NaN,NaT,NaN,NaN,NaN
4,20200630,3701132,동해면,"MULTIPOLYGON (((1181368.959 1784712.163, 11813...",포항시 남구,NaN,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
633,20200630,3840043,삼가면,"POLYGON ((1052540.942 1719533.002, 1052573.371...",합천군,2321.0,2011-02-07,경남,합천,삼가
633,20200630,3840043,삼가면,"POLYGON ((1052540.942 1719533.002, 1052573.371...",합천군,2369.0,2011-04-01,경남,합천,삼가
634,20200630,3840044,가회면,"POLYGON ((1044199.527 1722837.244, 1044218.563...",합천군,NaN,NaT,NaN,NaN,NaN
635,20200630,3840045,대병면,"POLYGON ((1046756.129 1730112.827, 1046808.008...",합천군,NaN,NaT,NaN,NaN,NaN


In [ ]:
result1["umd2"] = result1["sgg"] + " " + result1["ADM_DR_NM"]
result1

,BASE_DATE,ADM_DR_CD,ADM_DR_NM,geometry,sgg,index_right,date,OCUR_DO,OCUR_SGG,OCUR_EMD,umd2
0,20200630,3701111,구룡포읍,"MULTIPOLYGON (((1184734.239 1774617.254, 11847...",포항시 남구,NaN,NaT,NaN,NaN,NaN,포항시 남구 구룡포읍
1,20200630,3701112,연일읍,"POLYGON ((1160923.832 1786524.018, 1160962.498...",포항시 남구,2464.0,2013-03-10,경북,포항시,연일읍,포항시 남구 연일읍
2,20200630,3701113,오천읍,"POLYGON ((1172424.168 1778414.742, 1172426.383...",포항시 남구,2503.0,2013-05-13,경북,포항시남구,오천읍,포항시 남구 오천읍
3,20200630,3701131,대송면,"POLYGON ((1167538.928 1778439.770, 1167541.585...",포항시 남구,NaN,NaT,NaN,NaN,NaN,포항시 남구 대송면
4,20200630,3701132,동해면,"MULTIPOLYGON (((1181368.959 1784712.163, 11813...",포항시 남구,NaN,NaT,NaN,NaN,NaN,포항시 남구 동해면
...,...,...,...,...,...,...,...,...,...,...,...
633,20200630,3840043,삼가면,"POLYGON ((1052540.942 1719533.002, 1052573.371...",합천군,2321.0,2011-02-07,경남,합천,삼가,합천군 삼가면
633,20200630,3840043,삼가면,"POLYGON ((1052540.942 1719533.002, 1052573.371...",합천군,2369.0,2011-04-01,경남,합천,삼가,합천군 삼가면
634,20200630,3840044,가회면,"POLYGON ((1044199.527 1722837.244, 1044218.563...",합천군,NaN,NaT,NaN,NaN,NaN,합천군 가회면
635,20200630,3840045,대병면,"POLYGON ((1046756.129 1730112.827, 1046808.008...",합천군,NaN,NaT,NaN,NaN,NaN,합천군 대병면


In [ ]:
result_final = result1.drop(['BASE_DATE', "ADM_DR_CD", "ADM_DR_NM", "geometry", "sgg", "index_right", "OCUR_DO", "OCUR_SGG", "OCUR_EMD"], axis=1)
result_final

,date,umd2
0,NaT,포항시 남구 구룡포읍
1,2013-03-10,포항시 남구 연일읍
2,2013-05-13,포항시 남구 오천읍
3,NaT,포항시 남구 대송면
4,NaT,포항시 남구 동해면
...,...,...
633,2011-02-07,합천군 삼가면
633,2011-04-01,합천군 삼가면
634,NaT,합천군 가회면
635,NaT,합천군 대병면


In [ ]:
result_final.to_csv("fire_forest.csv")